# Đồ Án Cuối Kì Môn Khoa Học Dữ Liệu

**Giới thiệu thành viên**

- *Họ tên:* **Trần Ngọc Tịnh**       
  *MSSV:* **18120597**
  
  
  
  
- *Họ tên:* **Nguyễn Ngọc Năng Toàn**       
  *MSSV:* **18120600**

## Import 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

# You can also import other things ...
# YOUR CODE HERE (OPTION)

## Đọc dữ liệu từ file csv

In [2]:
data_df = pd.read_csv('data.csv', 
                      index_col=0) # Cho cột index là cột 0 (PassengerId)
data_df.head()

,confirmed,recovered,deaths,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city,lat,long,updated
Country,,,,,,,,,,,,,,,
Afghanistan,51526,41727,2191,35530081.0,652090.0,45.9,NaN,Asia,AF,Southern and Central Asia,4.0,Kabul,33.93911,67.709953,2021/01/01 15:22:42+00
Albania,58316,33634,1181,2930187.0,28748.0,71.6,NaN,Europe,AL,Southern Europe,8.0,Tirana,41.15330,20.168300,2021/01/01 15:22:42+00
Algeria,99610,67127,2756,41318142.0,2381741.0,69.7,800,Africa,DZ,Northern Africa,12.0,Alger,28.03390,1.659600,2021/01/01 15:22:42+00
Andorra,8049,7432,84,76965.0,468.0,83.5,"1,996",Europe,AD,Southern Europe,20.0,Andorra la Vella,42.50630,1.521800,2021/01/01 15:22:42+00
Angola,17553,11044,405,29784193.0,1246700.0,38.3,"1,112",Africa,AO,Central Africa,24.0,Luanda,-11.20270,17.873900,2021/01/01 15:22:42+00


In [3]:
data_df.index.duplicated().sum()

0

In [4]:
data_df['deaths'].dtype

dtype('int64')

In [6]:
data_df['deaths'].isna().sum()

0

In [9]:
data_df['deaths'].value_counts(normalize=True) * 100

0       7.558140
63      1.744186
1468    1.162791
7576    1.162791
104     1.162791
          ...   
1356    0.581395
335     0.581395
2641    0.581395
83      0.581395
76      0.581395
Name: deaths, Length: 150, dtype: float64

In [16]:
y_sr = data_df["deaths"] # sr là viết tắt của series
X_df = data_df.drop("deaths", axis=1)

In [37]:
X_df

,confirmed,recovered,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city,lat,long,updated
Country,,,,,,,,,,,,,,
Afghanistan,51526,41727,35530081.0,652090.0,45.9,NaN,Asia,AF,Southern and Central Asia,4.0,Kabul,33.939110,67.709953,2021/01/01 15:22:42+00
Albania,58316,33634,2930187.0,28748.0,71.6,NaN,Europe,AL,Southern Europe,8.0,Tirana,41.153300,20.168300,2021/01/01 15:22:42+00
Algeria,99610,67127,41318142.0,2381741.0,69.7,800,Africa,DZ,Northern Africa,12.0,Alger,28.033900,1.659600,2021/01/01 15:22:42+00
Andorra,8049,7432,76965.0,468.0,83.5,"1,996",Europe,AD,Southern Europe,20.0,Andorra la Vella,42.506300,1.521800,2021/01/01 15:22:42+00
Angola,17553,11044,29784193.0,1246700.0,38.3,"1,112",Africa,AO,Central Africa,24.0,Luanda,-11.202700,17.873900,2021/01/01 15:22:42+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Venezuela,113558,107583,31977065.0,912050.0,73.1,450,South America,VE,South America,862.0,Caracas,6.423800,-66.589700,2021/01/01 15:22:42+00
Vietnam,1474,1325,95540800.0,331689.0,69.3,398,Asia,VN,Southeast Asia,704.0,Hanoi,14.058324,108.277199,2021/01/01 15:22:42+00
Yemen,2099,1394,28250420.0,527968.0,59.8,999,Asia,YE,Middle East,887.0,Sanaa,15.552727,48.516388,2021/01/01 15:22:42+00


In [42]:
train_X, test_X, train_y, test_y = train_test_split(X_df, y_sr, test_size=0.3, 
                                                              stratify=y_sr, random_state=0)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.